In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Processing all images

In [2]:
%%time
images_True = []
images_False = []
image_dir = "data/images_small/traning/5-day/"
dirist = [x for x in os.listdir(image_dir) if not x.startswith(".")]
for folder_name in dirist:
    folder_dir = os.path.join(image_dir, folder_name)
    for file_name in os.listdir(folder_dir):
        file_dir = os.path.join(folder_dir, file_name)
        label = 1 if file_dir.strip().endswith("True.png") else 0
        img = cv2.imread(file_dir, cv2.IMREAD_GRAYSCALE)
        if label == 1:
            images_True.append(img)
        else:
            images_False.append(img)
images_True = np.array(images_True)
images_True = images_True / 255.0

images_False = np.array(images_False)
images_False = images_False / 255.0

print(f"TRUE: {len(images_True)} images with shape {images_True[0].shape}")
print(f"FALSE: {len(images_False)} images with shape {images_False[0].shape}")

TRUE: 107430 images with shape (32, 15)
FALSE: 85724 images with shape (32, 15)
CPU times: user 2.32 s, sys: 5.74 s, total: 8.05 s
Wall time: 13.7 s


In [3]:
import sys

def calculate_memory_usage(image_list):
    total_bytes = sys.getsizeof(image_list)  # Get the size of the list itself
    for img in image_list:
        total_bytes += sys.getsizeof(img)  # Add the size of each image array
    
    total_gb = total_bytes / (1024 ** 3)  # Convert bytes to gigabytes
    print(f"The total memory usage of the image list is approximately {total_gb:.3f} GB.")

# Example usage
calculate_memory_usage(images_True)
calculate_memory_usage(images_False)

The total memory usage of the image list is approximately 0.397 GB.
The total memory usage of the image list is approximately 0.317 GB.


In [4]:
import numpy as np
from sklearn.model_selection import train_test_split

# Step 1: Assign labels
true_labels = np.ones(len(images_True))  # Label 1 for True images
false_labels = np.zeros(len(images_False))  # Label 0 for False images

true_labels = true_labels[21706:]
images_True = images_True[21706:]

# Step 2: Combine images and labels
images = np.concatenate([images_True, images_False], axis=0)
labels = np.concatenate([true_labels, false_labels], axis=0)

# Step 3: Shuffle and split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.3, random_state=42, stratify=labels
)

# Output the results
print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Image shape: {X_train[0].shape}")

Training set: 120013 samples
Test set: 51435 samples
Image shape: (32, 15)


In [5]:
len(true_labels)

85724

In [6]:
len(false_labels)

85724

In [7]:
y_train.mean()

0.5000041662153266

In [8]:
from tensorflow.keras import models, layers
from tensorflow.keras.optimizers.legacy import Adam

def build_cnn_model(input_shape=(32, 15)):
    model = models.Sequential([
        # First Convolutional Block
        layers.Conv2D(64, (5, 3), padding='same', input_shape=(*input_shape, 1)),
        layers.LeakyReLU(alpha=0.1),  # Leaky ReLU with alpha=0.1
        layers.MaxPooling2D((2, 1)),
        
        # Second Convolutional Block
        layers.Conv2D(128, (5, 3), activation='relu', padding='same'),
        layers.LeakyReLU(alpha=0.1),  # Leaky ReLU with alpha=0.1
        layers.MaxPooling2D((2, 1)),  # Pooling with size (2, 1)
        
        # Flatten and Fully Connected Layers
        layers.Flatten(),
        layers.Dense(1, activation='sigmoid')  # Binary classification (True/False)
    ])
    
    # Use Adam optimizer with a learning rate of 0.1
    optimizer = Adam(learning_rate=0.1)
    
    # Compile the model
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [9]:
# Build the model
model = build_cnn_model()

# Summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 15, 64)        1024      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 32, 15, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 15, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 15, 128)       123008    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 16, 15, 128)       0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 8, 15, 128)        0         
 g2D)                                                   

In [10]:
history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=128
)

Epoch 1/20
751/751 [==============================] - 49s 65ms/step - loss: 2.0038 - accuracy: 0.5024 - val_loss: 0.6947 - val_accuracy: 0.5029
Epoch 2/20
751/751 [==============================] - 50s 66ms/step - loss: 0.6941 - accuracy: 0.5005 - val_loss: 0.6933 - val_accuracy: 0.4971
Epoch 3/20
751/751 [==============================] - 52s 69ms/step - loss: 0.6943 - accuracy: 0.4993 - val_loss: 0.7004 - val_accuracy: 0.5029
Epoch 4/20
751/751 [==============================] - 51s 68ms/step - loss: 0.6941 - accuracy: 0.5003 - val_loss: 0.6940 - val_accuracy: 0.4971
Epoch 5/20
751/751 [==============================] - 52s 69ms/step - loss: 0.6939 - accuracy: 0.5003 - val_loss: 0.6938 - val_accuracy: 0.5029
Epoch 6/20
751/751 [==============================] - 52s 70ms/step - loss: 0.6939 - accuracy: 0.4978 - val_loss: 0.6942 - val_accuracy: 0.5029
Epoch 7/20
751/751 [==============================] - 53s 70ms/step - loss: 0.6938 - accuracy: 0.5024 - val_loss: 0.6938 - val_accuracy:

KeyboardInterrupt: 

In [ ]:
model.save("my_model.h5")  # Save the model to a file

In [ ]:
from tensorflow.keras.models import load_model

loaded_model = load_model("my_model.h5")  # Load the model from the file